In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#pd.__version__

In [5]:
my_store = pd.HDFStore('test_store.h5')
my_store

<class 'pandas.io.pytables.HDFStore'>
File path: test_store.h5
/df                  frame        (shape->[6,2])                                                     
/toy_data            frame_table  (typ->appendable,nrows->10000,ncols->3,indexers->[index],dc->[B,C])

In [15]:
ts = pd.Series(index=['A','B','C','D'], data=np.random.randn(4))
df = pd.DataFrame(index=range(6), columns=['a','b'], data=np.random.random((6,2)))
my_store['ts'] = ts
my_store['df'] = df
my_store

<class 'pandas.io.pytables.HDFStore'>
File path: test_store.h5
/df            frame        (shape->[6,2])
/ts            series       (shape->[4])  

In [16]:
store_df = my_store['df']
(df == store_df).all()

a    True
b    True
dtype: bool

In [17]:
del my_store['ts']
my_store

<class 'pandas.io.pytables.HDFStore'>
File path: test_store.h5
/df            frame        (shape->[6,2])

In [25]:
n_rows = 10000
n_cols = 3
csv_path = 'toy_data.csv'
csv_file = open(csv_path, 'w')
csv_file.write("A B C\n")
for i in xrange(n_rows):
    for num in np.random.randn(3):
        csv_file.write(' '+str(num))
    csv_file.write('\n')
csv_file.close()

In [26]:
n_chunk = 1000
col_types = {'A':np.float,'B':np.float,'D':np.float}
reader = pd.read_csv(csv_path, sep=' ', dtype=col_types, index_col=False, chunksize=n_chunk, skipinitialspace=True)
first = True
data_cols = ['B','C']
for chunk in reader:
    if first:
        my_store.append('toy_data', chunk, data_columns=data_cols, index=False)
        first = False
    else:
        my_store.append('toy_data', chunk, index=False)
my_store

<class 'pandas.io.pytables.HDFStore'>
File path: test_store.h5
/df                  frame        (shape->[6,2])                                                     
/toy_data            frame_table  (typ->appendable,nrows->10000,ncols->3,indexers->[index],dc->[B,C])

In [27]:
my_store.create_table_index('toy_data', optlevel=9, kind='full')

In [24]:
del my_store['toy_data']

In [21]:
##########################################

In [6]:
#Problem 1
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
store = pd.HDFStore('Election.h5',mode='r')

In [7]:
#del store['Election']
#store.open()
print store
#del store['Election']

<class 'pandas.io.pytables.HDFStore'>
File path: Election.h5
/Election            frame_table  (typ->appendable,nrows->6036458,ncols->18,indexers->[index],dc->[cand_nm,contbr_st,contb_receipt_amt,contbr_occupation,contb_receipt_dt])


In [8]:
store.close()
store.open()
del store['Election']

HDF5ExtError: HDF5 error back trace

  File "..\..\hdf5-1.8.11\src\H5L.c", line 609, in H5Ldelete
    unable to delete link
  File "..\..\hdf5-1.8.11\src\H5L.c", line 2344, in H5L_delete
    can't unlink object
  File "..\..\hdf5-1.8.11\src\H5Gtraverse.c", line 861, in H5G_traverse
    internal path traversal failed
  File "..\..\hdf5-1.8.11\src\H5Gtraverse.c", line 641, in H5G_traverse_real
    traversal operator failed
  File "..\..\hdf5-1.8.11\src\H5L.c", line 2301, in H5L_delete_cb
    unable to remove link from group
  File "..\..\hdf5-1.8.11\src\H5Gobj.c", line 1020, in H5G_obj_remove
    can't remove object
  File "..\..\hdf5-1.8.11\src\H5Gstab.c", line 386, in H5G__stab_remove
    unable to remove entry
  File "..\..\hdf5-1.8.11\src\H5B.c", line 1676, in H5B_remove
    unable to remove entry from B-tree
  File "..\..\hdf5-1.8.11\src\H5B.c", line 1402, in H5B_remove_helper
    key not found in leaf node
  File "..\..\hdf5-1.8.11\src\H5Gnode.c", line 872, in H5G_node_remove
    u

In [9]:
store.close()

In [10]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: Election.h5
File is CLOSED

In [3]:


csv_path = 'Election.csv'
n_chunk = 50000
col_types = {'cmte_id':np.object,
             'cand_id':np.object,
             'cand_nm':np.object,
             'contbr_nm':np.object,
             'contbr_city':np.object,
             'contbr_st':np.object,
             'contbr_zip':np.object,
             'contbr_employer':np.object,
             'contbr_occupation':np.object,
             'contb_receipt_amt':np.float,
             'receipt_desc':np.object,
             'memo_cd':np.object,
             'memo_text':np.object,
             'form_tp':np.object,
             'file_num':np.float,
             'tran_id':np.object,
             'election_tp':np.object
             }

In [4]:
reader = pd.read_csv(csv_path,
                     dtype=col_types,
                     index_col=False,
                     parse_dates=['contb_receipt_dt'],
                     chunksize=n_chunk
                     #skipinitialspace=True
                     )

In [5]:
first = True
data_cols = ['cand_nm','contbr_st','contb_receipt_amt','contbr_occupation','contb_receipt_dt']
chunk_num = 1
for chunk in reader:
    print chunk_num
    chunk_num+=1
    if first:
        store.append('Election', chunk, data_columns=data_cols, index=False, min_itemsize={'values':50})
        first = False
    else:
        store.append('Election', chunk, index=False)
store

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


<class 'pandas.io.pytables.HDFStore'>
File path: Election.h5
/Election            frame_table  (typ->appendable,nrows->6036458,ncols->18,indexers->[index],dc->[cand_nm,contbr_st,contb_receipt_amt,contbr_occupation,contb_receipt_dt])

In [6]:
store.create_table_index('Election', optlevel=9, kind='full')

In [6]:
my_store.select('toy_data', where = ["'B'<0","'C'>0"])

,A,B,C
5,-0.117438,-1.158629,0.802489
7,-1.149631,-1.015535,0.089663
8,-1.176948,-2.498150,1.169316
9,0.053147,-0.275624,0.567064
11,-0.015035,-0.551113,0.925186
14,-0.841065,-0.516207,0.132337
16,-2.596223,-0.972695,1.265653
23,0.677981,-1.550332,0.074337
29,-1.167706,-0.789103,1.112683
30,-0.612456,-1.898663,0.183705


In [2]:
cands = store.select_column('Election', 'cand_nm').unique()
cands

array(['Bachmann, Michele', 'Romney, Mitt', 'Obama, Barack',
       "Roemer, Charles E. 'Buddy' III", 'Pawlenty, Timothy',
       'Johnson, Gary Earl', 'Paul, Ron', 'Santorum, Rick', 'Cain, Herman',
       'Gingrich, Newt', 'McCotter, Thaddeus G', 'Huntsman, Jon',
       'Perry, Rick', 'Stein, Jill'], dtype=object)

In [3]:
n_UT = len(store.select('Election', where = ["'contbr_st'=='UT'", "columns='contbr_st'"]))
n_UT

69125

In [4]:
n_rom_UT = len(store.select('Election', where = ["'contbr_st'=='UT'","'cand_nm'=='Romney, Mitt'"]))#, "columns='cand_nm'"]))
n_rom_UT

45628

In [5]:
np.float(n_rom_UT)/n_UT

0.6600795660036166

In [6]:
#Problem 2
n_CA_BO = len(store.select('Election', where=["'contbr_st'=='CA'", "'cand_nm'=='Obama, Barack'"]))
n_CA_MR = len(store.select('Election', where=["'contbr_st'=='CA'", "'cand_nm'=='Romney, Mitt'"]))
print "CA Obama Contributors:", n_CA_BO
print "CA Romney Contributors:", n_CA_MR

CA Obama Contributors: 724300
CA Romney Contributors: 193123


In [7]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: Election.h5
/Election            frame_table  (typ->appendable,nrows->6036458,ncols->18,indexers->[index],dc->[cand_nm,contbr_st,contb_receipt_amt,contbr_occupation,contb_receipt_dt])

In [8]:
states = store.select_column('Election','contbr_st')
states = states.value_counts()
states

In [9]:
st_contb = []
i = 1
for state in states.index:
    print i
    i+=1
    grp = store.select('Election', where=["'contbr_st'=\"{}\"".format(state), "columns='contb_receipt_amt'"])
    st_contb.append(grp['contb_receipt_amt'].mean())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


In [10]:
state_info = pd.DataFrame({'Count':states, 'Avg Contb':st_contb})
state_info.sort(columns='Count', ascending=False, inplace=True)
plt.subplot(211)
state_info['Count'][:10].plot(kind='bar')
plt.subplot(212)
state_info['Avg Contb'][:10].plot(kind='bar')
plt.show()
plt.close()

In [12]:
cand = store.select_column('Election','cand_nm')
cand = cand.value_counts()
cand

Obama, Barack                     4117404
Romney, Mitt                      1593686
Paul, Ron                          151722
Santorum, Rick                      52098
Gingrich, Newt                      51795
Cain, Herman                        20160
Perry, Rick                         13605
Bachmann, Michele                   13240
Johnson, Gary Earl                   6599
Roemer, Charles E. 'Buddy' III       6010
Pawlenty, Timothy                    4555
Huntsman, Jon                        4193
Stein, Jill                          1317
McCotter, Thaddeus G                   74
dtype: int64

In [14]:
cand.index

Index([u'Obama, Barack', u'Romney, Mitt', u'Paul, Ron', u'Santorum, Rick', u'Gingrich, Newt', u'Cain, Herman', u'Perry, Rick', u'Bachmann, Michele', u'Johnson, Gary Earl', u'Roemer, Charles E. 'Buddy' III', u'Pawlenty, Timothy', u'Huntsman, Jon', u'Stein, Jill', u'McCotter, Thaddeus G'], dtype='object')

In [15]:
cand2 = store.select_column('Election','cand_nm').unique()
cand2

array(['Bachmann, Michele', 'Romney, Mitt', 'Obama, Barack',
       "Roemer, Charles E. 'Buddy' III", 'Pawlenty, Timothy',
       'Johnson, Gary Earl', 'Paul, Ron', 'Santorum, Rick', 'Cain, Herman',
       'Gingrich, Newt', 'McCotter, Thaddeus G', 'Huntsman, Jon',
       'Perry, Rick', 'Stein, Jill'], dtype=object)

In [3]:
grp = store.select('Election', where=["'cand_nm'='Romney, Mitt'", "columns='contb_receipt_amt'"])
grp

,contb_receipt_amt
411,200
412,100
413,225
414,126
415,225
416,200
417,100
418,126
419,110
420,41


In [4]:
print grp['contb_receipt_amt'].sum()

653630745.69


In [ ]:
store.open()
cand = store.select_column('Election','cand_nm').unique()
cand_cont = []
i = 1
for candidate in cand:
    print i
    i+=1
    grp = store.select('Election', where=["'cand_nm'==candidate", "columns=='contb_receipt_amt'"])
    TOT = grp['contb_receipt_amt'].sum()
    print candidate, TOT
    cand_cont.append(TOT)
    


1
Bachmann, Michele

In [10]:
store.open()
print cand_cont
print cand
store.close()

[2685817.79, 653630745.68999958, 548880098.67000055, 369324.53999999998, 4255054.0899999999, 2637706.3800000004, 21650581.039999999, 10917859.82, 6965980.9900000002, 11816319.830000002, 37030.0, 3156942.3999999999, 18409641.91, 334934.08999999997]
['Bachmann, Michele' 'Romney, Mitt' 'Obama, Barack'
 "Roemer, Charles E. 'Buddy' III" 'Pawlenty, Timothy' 'Johnson, Gary Earl'
 'Paul, Ron' 'Santorum, Rick' 'Cain, Herman' 'Gingrich, Newt'
 'McCotter, Thaddeus G' 'Huntsman, Jon' 'Perry, Rick' 'Stein, Jill']


In [7]:
store.close()

In [11]:
store.open()
contribs = pd.Series(index = cand, data = cand_cont)
contribs.plot(kind='barh')
plt.plot()
plt.savefig('cand_contb.pdf')
plt.clf()
store.close()

In [ ]:
store.open()
cand2 = store.select_column('Election','cand_nm').unique()
cand2_cont = []
i = 1
for candidate2 in cand2:
    print i
    i+=1
    grp = store.select('Election', where=["'cand_nm'=\"{}\"".format(candidate2), "columns='contb_receipt_amt'"])
    TOT = grp['contb_receipt_amt'].sum()
    print candidate2, TOT
    cand_cont2.append(TOT)